In [14]:
import requests
import hmac
import hashlib
import time
from urllib.parse import urlparse
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get API credentials from environment variables
BITCOINDE_API_KEY = os.getenv('BITCOINDE_API_KEY')
BITCOINDE_API_SECRET = os.getenv('BITCOINDE_API_SECRET')

In [16]:
def get_bitcoinde_signature(method, url, nonce, post_params=''):
    # Extract path from full URL
    path = urlparse(url).path
    
    # Create signature string according to Bitcoin.de documentation
    hmac_data = '#'.join([
        method.upper(),
        path,  # Use path instead of full URL
        BITCOINDE_API_KEY,
        nonce,
        post_params
    ])
    
    try:
        signature = hmac.new(
            BITCOINDE_API_SECRET.encode('utf-8'),
            hmac_data.encode('utf-8'),
            hashlib.sha256
        ).hexdigest()
        return signature
    except Exception as e:
        print(f"Error generating signature: {e}")
        return None

def get_bitcoinde_orderbook():
    url = 'https://api.bitcoin.de/v4/btceur/orderbook'
    method = 'GET'
    nonce = str(int(time.time() * 1000))  # Use millisecond precision
    signature = get_bitcoinde_signature(method, url, nonce)
    
    if not signature:
        return None
    
    headers = {
        'X-API-KEY': BITCOINDE_API_KEY,
        'X-API-NONCE': nonce,
        'X-API-SIGNATURE': signature
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching orderbook: {e}")
        if response.text:
            print(f"Response: {response.text}")
        return None

# Fetch Bitcoin.de orderbook
bitcoinde_orders = get_bitcoinde_orderbook()
if bitcoinde_orders and 'asks' in bitcoinde_orders and 'bids' in bitcoinde_orders:
    bitcoinde_best_ask = float(bitcoinde_orders['asks'][0]['price'])
    bitcoinde_best_bid = float(bitcoinde_orders['bids'][0]['price'])
    print(f"Best ask: {bitcoinde_best_ask}")
    print(f"Best bid: {bitcoinde_best_bid}")
else:
    print("Failed to fetch orderbook data")

Error fetching orderbook: 400 Client Error: Bad Request for url: https://api.bitcoin.de/v4/btceur/orderbook
Response: {"errors":[{"message":"Invalid signature","code":5}],"credits":18}
Failed to fetch orderbook data


In [13]:
# Fetch Binance BTC/EUR price directly
try:
    binance_response = requests.get('https://api.binance.com/api/v3/ticker/price?symbol=BTCEUR')
    binance_response.raise_for_status()
    binance_price_eur = float(binance_response.json()['price'])
except Exception as e:
    print(f"Error fetching Binance price: {e}")
    exit()

print(f'Current Bitcoin Price: €{binance_price_eur:,.2f} EUR')

Current Bitcoin Price: €76,275.99 EUR


In [ ]:
print(f'Bitcoin.de Best Ask: €{bitcoinde_best_ask:,.2f}')
print(f'Bitcoin.de Best Bid: €{bitcoinde_best_bid:,.2f}')
print(f'Binance Price (EUR): €{binance_price_eur:,.2f}')

In [ ]:
# Calculate price differences
diff_ask = ((bitcoinde_best_ask - binance_price_eur) / binance_price_eur) * 100
diff_bid = ((bitcoinde_best_bid - binance_price_eur) / binance_price_eur) * 100

print(f'\nPrice difference (Ask): {diff_ask:,.2f}%')
print(f'Price difference (Bid): {diff_bid:,.2f}%')